<a href="https://colab.research.google.com/github/prajwal467/CE888-Assignment2/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import gensim

import nltk as nl
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score,f1_score

from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras as keras
from tensorflow.python.keras.models import Sequential
import tensorflow.python.keras.layers as layers

In [ ]:
path = "proccessed_data/emotion/"

In [ ]:
# Function to map words to their respective values 
def prepare_text(tokens):
    text = []
    max_len = 0
    for sentence in tokens:
        line = []
        for word in sentence:
            try:
                line.append(w2v[word])
            except exception as e:
                print(e)
                print(word)
                line.append(0)
        max_len = max(max_len,len(line))       
        text.append(line)
    return np.array(text),max_len

In [ ]:
#Reading datasets
data_train = pd.read_csv(path+"train_text.csv")
data_test = pd.read_csv(path+"test_text.csv")
data_val = pd.read_csv(path+"val_text.csv")

FileNotFoundError: ignored

In [ ]:
train_label = pd.read_csv(path+"train_labels.csv")
test_label = pd.read_csv(path+"test_labels.csv")
val_label = pd.read_csv(path+"val_labels.csv")

In [ ]:
data_val.shape,val_label.shape

((374, 1), (374, 1))

# Word2Vec

In [ ]:
# Splitting sentences into words

train_tokens = [nl.word_tokenize(sentences) for sentences in data_train.text]
test_tokens = [nl.word_tokenize(sentences) for sentences in data_test.text]
val_tokens = [nl.word_tokenize(sentences) for sentences in data_val.text]

In [ ]:
#training word2vec model on complete vocabulary (train + test vocabulary)

model = gensim.models.Word2Vec(size=250, min_count=1, workers=-1)
model.build_vocab((train_tokens+test_tokens+val_tokens))
model.train((train_tokens+test_tokens+val_tokens),total_examples= len(train_tokens+test_tokens+val_tokens),epochs = 2500)

(0, 0)

In [ ]:
# storing trained values of every word [0:100] --> [1] 
w2v = dict(zip(model.wv.index2word, np.mean(model.wv.syn0,axis=1)))

C:\Users\soura\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [ ]:
# Converting strinf to float values and getting max_len for padding
X_train,max_len = prepare_text(train_tokens)
X_test,_ = prepare_text(test_tokens)
X_val,_ = prepare_text(val_tokens)

In [ ]:
# Padding to ensure dimensions

X_train = pad_sequences(X_train,dtype='float32',padding='post',maxlen= max_len)
X_test = pad_sequences(X_test,dtype='float32',padding='post',maxlen = max_len )
X_val = pad_sequences(X_val,dtype='float32',padding='post',maxlen = max_len )

In [ ]:
# converting labels to one-hot-vector and categorical-vector
oe_enc = OneHotEncoder()

Y_train = oe_enc.fit_transform(np.array(train_label.text).reshape(-1,1)).toarray()
Y_test = oe_enc.transform(np.array(test_label.text).reshape(-1,1)).toarray()
Y_val = oe_enc.transform(np.array(val_label.text).reshape(-1,1)).toarray()

true_label_train = np.argmax(Y_train,axis=1)
true_label_test = np.argmax(Y_test,axis=1)
true_label_val = np.argmax(Y_val,axis=1)

In [ ]:
# storing true dimensions
n_len,features = X_train.shape
n_len_test,features = X_test.shape
n_len_val,features = X_val.shape

# LSTM

In [ ]:
# defining optimizer and callback
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=4)

In [ ]:
X_train,X_test = X_train.reshape(n_len,features,1),X_test.reshape(n_len_test,features,1)
X_val = X_val.reshape(n_len_val,features,1)

In [ ]:
model = Sequential(name="LSTM")
model.add(layers.LSTM(16,input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(layers.LSTM(56,return_sequences=True,dropout=0.5))
model.add(layers.LSTM(128))
model.add(layers.Dense(len(np.unique(true_label_train)),activation="softmax"))

In [ ]:
model.compile(optimizer = optimizer,loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),callbacks=[es],epochs=50,verbose=1,batch_size=32)

Epoch 1/50
102/102 [==============================] - 3s 27ms/step - loss: 1.2873 - accuracy: 0.4292 - val_loss: 1.2883 - val_accuracy: 0.3927
Epoch 2/50
102/102 [==============================] - 1s 11ms/step - loss: 1.2656 - accuracy: 0.4298 - val_loss: 1.2841 - val_accuracy: 0.3927
Epoch 3/50
102/102 [==============================] - 1s 12ms/step - loss: 1.2657 - accuracy: 0.4298 - val_loss: 1.2822 - val_accuracy: 0.3927
Epoch 4/50
102/102 [==============================] - 1s 11ms/step - loss: 1.2663 - accuracy: 0.4298 - val_loss: 1.2822 - val_accuracy: 0.3927
Epoch 5/50
102/102 [==============================] - 1s 11ms/step - loss: 1.2674 - accuracy: 0.4298 - val_loss: 1.2829 - val_accuracy: 0.3927
Epoch 00005: early stopping


In [ ]:
print("F1-SCORE",f1_score(true_label_val,np.argmax(model.predict(X_val),axis=1),average="micro"))

F1-SCORE 0.42780748663101603
